In [3]:
from selenium import webdriver
from chromedriver_py import binary_path
from auth_data import aquaPassword, aquaLogin
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()

options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36' )

prefs = {'profile.managed_default_content_settings.images':2}
options.add_experimental_option('prefs',prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

driver.get("https://catalog.aquamarine.kz/catalog/index.aspx")
time.sleep(3)

email_input = driver.find_element(By.ID, 'ctl02_tbEmail')
email_input.clear()
email_input.send_keys(aquaLogin)
time.sleep(0.5)
password_input = driver.find_element(By.ID, 'ctl02_tbPassword')
password_input.clear()
password_input.send_keys(aquaPassword)
time.sleep(0.5)
submit_button =driver.find_element(By.ID, 'ctl02_btnLogin').click()
time.sleep(2)


WebDriverWait(driver, 120).until(EC.invisibility_of_element((By.ID, "loading")))


input = 5                 # choose type





if (input==5):
        coating = 'avid-5'
        ifgold = ''
        addarticle = '.5'
        addname = '' 
if (input==6):
        coating = 'avid-6'
        ifgold = '&av=6' 
        addarticle = '.6'
        addname = 'с позолотой ' 
  

checkbox = driver.find_element_by_id(coating).click() 
time.sleep(0.5)

WebDriverWait(driver, 120).until(EC.invisibility_of_element((By.ID, "loading")))






AttributeError: 'WebDriver' object has no attribute 'find_element_by_id'

In [27]:
driver.switch_to.window(driver.window_handles[0])
pagecount


170

In [4]:
male = 'Женские'
probe = '925'

if input == 6:
    coat ='Да'
else:
    coat = ''



def takearticlefromthename(name):
  a = name.split(' ')
  return a[2]

def clean(a):
  b=[]
  for i in a:
     
    b.append(i.replace('Среднийвес','').replace('Заграмм',' ').replace('KZT','').replace('Цена','').replace('Наличие','').replace('Артикул','').replace('Металл','').strip())
     
  return b


def addsclean(str):
    index = str.find(' ')
    res_str = ''.join([str[i] for i in range(len(str)) if i >= index])
    str =res_str
  
    for d in '1234567890':
        str=str.replace(d, '')
 
    str= str.split(';')
 
    str3=[] 
    for each in str:
        str3.append(each.strip())
      
            
    str2 = []
    for item in str3: 
        if item not in str2:
            str2.append(item)
        
 
    str2 =','.join(str2)
    return str2
try:
    data = [] 
    soup = BeautifulSoup(driver.page_source, 'lxml')
    pages = soup.findAll('div', class_='page')
    pagedivs = int((len(pages))/2)
     
    
    if pagedivs < 12:
        pagecount = pagedivs
        
    else:
        lastpagenumber = driver.find_element(By.XPATH, f'//*[@id="pane"]/table[1]/tbody/tr/td[2]/div/div[12]').text
        pagecount  = int(lastpagenumber) 
     
    #pagecount = 143
    for i in range(1,pagecount+1):
        
        seconds = time.time()
        local_time = time.ctime(seconds)
        print("Local time:", local_time, 'page', i, 'of', pagecount)
        
        if i < 8:
            i = str(i)
        else:
            i = '8'

        click = driver.find_element(By.XPATH, f'//*[@id="pane"]/table[1]/tbody/tr/td[2]/div/div[{i}]').click()
        time.sleep(1)
        WebDriverWait(driver,120).until(EC.invisibility_of_element((By.ID, "loading")))
        #time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'lxml')
        items = soup.findAll("div", class_ = 'item') 

        driver.execute_script(f'''window.open("","_blank");''')
        #time.sleep(0.5)
        driver.switch_to.window(driver.window_handles[1])
        #time.sleep(0.5)
        for item in items :

            imagelink ='https://catalog.aquamarine.kz' + item.find('div', class_ = 'image').find('img')['src']
            article = item.find("div", class_ = 'art').text+addarticle
            itemlink = 'https://catalog.aquamarine.kz' + item.find('a', class_ = 'pfancy pfancy.iframe plink').get('href')
            hr = item.find('a', class_ = 'pfancy pfancy.iframe plink').get('href') 
            #driver.execute_script(f'''window.open("{itemlink+ifgold}","_blank");''')

            driver.get(itemlink+ifgold)

            #driver.find_element_by_xpath('//a[@href="'+hr+'"]').click()

            itemproperties = BeautifulSoup(driver.page_source,'lxml')
            
            itemdata = itemproperties.find('div', class_ = 'productinfo').findAll('tr', class_ = 'data')
            idata = [] 
            for each in itemdata : 
                idata.append(each.text.replace('\xa0','').replace('\n',' ').replace('KZT','').strip())
            for each in idata :
                if (each.find('Коллекция')==0):
                    idata.remove(each)
            for each in idata :       
                if (each.find('Вставки')==0):
                    idata[len(idata)-1] = addsclean(idata[len(idata)-1])
                    
                    adds = addsclean(each)
                else:
                    
                    adds = ''
            cleaned =  idata[4].replace('Цена','').strip()   #seiling price
            
            try:
                price  = int(cleaned)

                if price < 1000:
                    idata[0] = '2500'

                elif price<3000 :
                    idata[0] = str(round((price * 3),-2))

                else:
                    idata[0] = str(round((price * 2 + 1500),-2))

                cleaned2 =  idata[4].replace('Цена','').strip()   #wholesale price
                price  = int(cleaned2)

                idata[1] = str(round((price * 0.3 + price),-2))
            except:
                idata[1] = ''
            
                
            
 
            try:
                diamonds = itemproperties.find('div', class_ = 'productinfo').findAll('table', class_ = 'assetinfo')
                if len(diamonds)>0:
                    diamonds = diamonds[0].findAll('tr')
                     
                    props = diamonds[1].findAll('td')
                    if props[2].text =='-' and len(diamonds)>2:
                        idata[5] = ''
                        name = itemproperties.find("div", class_ = 'header').text.split(' ')[0] + ' AQUAMARINE ' + article+ " серебро " + addname + adds.lower()

                        main = [name,article,itemlink, imagelink,probe, male,coat]

                        sum =[]
                        idata = clean(idata)
                        for i in main:
                            sum.append(i)
                        for i in idata:
                            sum.append(i)

                        data.append(sum) 
                        

                else:
                    sizes = itemproperties.find('div', class_ = 'productinfo').findAll('div', class_ = 'center ssize')
                    for each in sizes:
                        size = each.find("span", class_= 'qtyi').text
                        sizecount = each.find("span", class_= 'gray qtyi').text
                        sizecount = sizecount.replace('/','') 
                        if (size !='0'):

                            if int(sizecount) > 1:


                                if (itemproperties.find("div", class_ = 'header').text.split(' ')[0] == 'Кольцо'):
                                    unit = 'р'
                                else:
                                    unit = 'см'
                                namesize = size.replace(',', '-')
                                name = itemproperties.find("div", class_ = 'header').text.split(' ')[0] + ' AQUAMARINE '+ article  + " серебро " + addname + adds.lower()  + f' {size}' + unit  
                                art2 = takearticlefromthename(name)+ f'_{namesize}'
                                idata[5] = size

                                main = [name,art2,itemlink, imagelink,probe, male,coat]

                                sum =[]
                                idata = clean(idata)
                                for i in main:
                                    sum.append(i)
                                for i in idata:
                                    sum.append(i)
                                data.append(sum) 
                        else:
                            if int(sizecount) > 1:

                                name = itemproperties.find("div", class_ = 'header').text.split(' ')[0] + ' AQUAMARINE ' +article + " серебро " + addname + adds.lower()
                                art2 = article    
                                idata[5] = ''

                                main = [name,art2,itemlink, imagelink,probe, male,coat]

                                sum =[]
                                idata = clean(idata)
                                for i in main:
                                    sum.append(i)
                                for i in idata:
                                    sum.append(i)

                                data.append(sum)     
            except: 
                print('exception')
                #if int(sizecount) > 1:
                #name = itemproperties.find("div", class_ = 'header').text.split(' ')[0] + ' AQUAMARINE ' + article+ " серебро " + addname + adds.lower()

                #main = [name,article,itemlink, imagelink,probe, male,coat]

                #sum =[]
                #idata = clean(idata)
                #for i in main:
                    #sum.append(i)
                #for i in idata:
                    #sum.append(i)

                #data.append(sum)    

        driver.close() 
        driver.switch_to.window(driver.window_handles[0])
    time.sleep(0.5)
except Exception as ex:
    
    print(ex)
    
    df = pd.DataFrame(data)
    
    try:
        df.to_excel('aquaSilver.xlsx')
        local_time = time.ctime(seconds)
        print("Local time:", local_time, 'done with exception')
    except:

        df.to_excel('aquaSilver-.xlsx')
        local_time = time.ctime(seconds)
        print("Local time:", local_time, 'done with exception')
 
df = pd.DataFrame(data)
try:
    df.to_excel('aquaSilver.xlsx')
    local_time = time.ctime(seconds)
    print("Local time:", local_time, 'done')
except:

    df.to_excel('aquaSilver-.xlsx')
    local_time = time.ctime(seconds)
    print("Local time:", local_time, 'done')

Local time: Wed May 10 11:03:38 2023 page 1 of 222
Local time: Wed May 10 11:03:43 2023 page 2 of 222
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.140)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0037DCE3+50899]
	(No symbol) [0x0030E111]
	(No symbol) [0x00215588]
	(No symbol) [0x001FD333]
	(No symbol) [0x0025F4DB]
	(No symbol) [0x0026DB33]
	(No symbol) [0x0025B6F6]
	(No symbol) [0x00237708]
	(No symbol) [0x0023886D]
	GetHandleVerifier [0x005E3EAE+2566302]
	GetHandleVerifier [0x006192B1+2784417]
	GetHandleVerifier [0x0061327C+2759788]
	GetHandleVerifier [0x00415740+672048]
	(No symbol) [0x00318872]
	(No symbol) [0x003141C8]
	(No symbol) [0x003142AB]
	(No symbol) [0x003071B7]
	BaseThreadInitThunk [0x7636FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77387A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77387A6E+238]

Local time: Wed May 10 11:03:43 2023 done with exception
Local time: Wed May 10 11:03:43 20

In [5]:
driver.switch_to.window(driver.window_handles[0])
pagecount


21

In [ ]:

df = pd.DataFrame(data)
try:
    df.to_excel('aquaSilver.xlsx')
except:

    df.to_excel('aquaSilver-.xlsx')
    

In [81]:

itemproperties = BeautifulSoup(driver.page_source,'lxml')

itemdata = itemproperties.find('div', class_ = 'productinfo').findAll('tr', class_ = 'data')

diamonds = itemproperties.find('div', class_ = 'productinfo').findAll('table', class_ = 'assetinfo')
diamonds = diamonds[0].findAll('tr')
len(diamonds)

   
 

2